In [ ]:
import yt_dlp

url = 'https://www.bilibili.com/video/BV1BU4y1Z7dP'


In [ ]:
ydl_opts = {
    'outtmpl': '%(title)s.%(ext)s', 
    'format': 'bestaudio',           
    'ffmpeg_location': r'C:\Users\rekxh\Downloads\ffmpeg-7.1.1-full_build\ffmpeg-7.1.1-full_build\bin\ffmpeg.exe',
    'postprocessors': [{             # 转换为 mp3 或其他格式（可选）
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',     # 你也可以改成 'm4a', 'wav', 等
        'preferredquality': '192',
    }],
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

In [ ]:
## mp4 version available

# ydl_opts = {
#     'outtmpl': '%(title)s.%(ext)s',
#     'format': 'bestvideo+bestaudio/best',
#     'merge_output_format': 'mp4',
#     'ffmpeg_location': r'C:\Users\rekxh\Downloads\ffmpeg-7.1.1-full_build\ffmpeg-7.1.1-full_build\bin\ffmpeg.exe',
# }

# with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#     ydl.download([url])

In [ ]:
!pip install pydub


In [ ]:
from pathlib import Path
from pydub import AudioSegment

CHUNK_MS = 3 * 60 * 1000   # 3 分钟 → 毫秒

def slice_mp3(file_path: Path, out_dir: Path = None):
    """
    把单个 MP3 切成 3 分钟段。
    参数:
        file_path: 目标 MP3
        out_dir  : 片段保存的目录，默认为同级 /chunks
    """
    audio = AudioSegment.from_mp3(file_path)
    out_dir = out_dir or file_path.with_suffix('').parent / 'chunks'
    out_dir.mkdir(exist_ok=True)

    for i in range(0, len(audio), CHUNK_MS):
        chunk = audio[i:i + CHUNK_MS]
        # 如果最后一段不足 3 分钟，也照切——毕竟没人嫌披萨片太小
        chunk_name = f"{file_path.stem}_part{i // CHUNK_MS + 1:03d}.mp3"
        chunk.export(out_dir / chunk_name, format="mp3")

if __name__ == "__main__":
    # 把当前目录下所有 MP3 都切了，像给迷路的灵魂安上翅膀
    for mp3 in Path(".").glob("*.mp3"):
        slice_mp3(mp3)
    print("切片完成，碎片已就绪。")
